In [3]:
import os
import re
import subprocess


# name of channel to be scanned
channel_prefix = "https://www.youtube.com/"
channel = "@StronnyCuttles"

os.mkdir(channel)
os.mkdir(f"{channel}/downloaded")
os.mkdir(f"{channel}/processed")
os.mkdir(f"{channel}/transcripts")


# file containing keywords
keywords_file = "keywords.txt"

# file containing thematic searches
thematic_file = "thematic.txt"

downloaded = []
downloaded_file = f"downloaded_{channel}.txt"
if os.path.exists(downloaded_file):
    with open(downloaded_file, "r") as f:
        downloaded = f.read().splitlines()
else:
    with open(downloaded_file, "w") as f:
        pass

# scan channel for videos
# live videos
os.system(f"yt-dlp --get-id --skip-download {channel_prefix+channel} > {channel}/videos.txt")
videos = []
with open(f"channel_content_{channel}.txt", "r") as f:
    for line in f:
        if line.startswith("Error"):
            continue
        videos.append(line.strip())


# compare list of videos to videos already downloaded
to_download = list(set(videos) - set(downloaded))
    

In [4]:
newly_downloaded = []
for video in to_download:
    print(f"Downloading {video}")
    output = os.system(f"yt-dlp -f bestaudio -P {channel}/{downloaded} {channel_prefix}watch?v={video}")
    if output==0:
        with open(downloaded_file, "a") as f:
            f.write(video + "\n")
    else:
        print(f"Error downloading {video}")

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [10]:
f"whisper \"{channel}/downloaded/{downloaded_files[0]}\" --device cuda"

'whisper "@StronnyCuttles/downloaded/girlfriend begs you to let her hold it asmr [n9V9pF55XOI].webm" --device cuda'

In [14]:
#collect file names in {channel}/downloaded
downloaded_files = os.listdir(f"{channel}/downloaded")
for file in downloaded_files:
    if file.endswith(".webm"):
        result = os.system(f"whisper \"{channel}/downloaded/{file}\" --device cuda --output_dir \"{channel}/transcripts/\"")
        if result==0:
            #move file to {channel}/processed
            os.rename(f"{channel}/downloaded/{file}", f"{channel}/processed/{file}")